In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as spec
import pandas as pd
from astropy.io import ascii
import scipy.stats as stats
from ipywidgets import FloatProgress
from IPython.display import display
import requests
import astropy.units as u
from astropy.coordinates import SkyCoord

Get the KOI catalog from the archive (if we don't already have it)

In [2]:
if False:
    urlDr25Koi = "https://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=q1_q17_dr25_koi&select"

    r = requests.get(urlDr25Koi)
    if r.status_code != requests.codes.ok:
        r.raise_for_status()
    fh = StringIO(r.content)
    dr25Koi = pd.read_csv(fh, dtype={"kepoi_name":str})
    dr25Koi.to_csv("dr25_kois_archive.txt", index=False)
else:
    dr25Koi = pd.read_csv("dr25_kois_archive.txt", dtype={"kepoi_name":str})

print("Loaded " + str(len(dr25Koi)) + " KOIs")



Loaded 8054 KOIs


Get the certified false positive table.  We have to download it manually.

In [3]:
cfpTableName = "fpwg_2019.09.16_18.35.55.csv"
archiveCfpTable = pd.read_csv(cfpTableName, dtype={"kepoi_name":str}, header=71)


Get the DR25 stellar catalog with updates from Savita's supplement

In [4]:
starlist = "dr25_stellar_updated_feh_all.txt"
kic = pd.read_csv(starlist)


Now we'll merge these tables together, starting by merging the cfpTable and dr25Koi, using kepoi_name as the key.

In [5]:
cfpTable = pd.merge(archiveCfpTable, dr25Koi, on="kepoi_name", how="inner")
# the only column common to both is kepid, so rename kepid_x to kepid and drop kepid_y
cfpTable["kepid"] = cfpTable["kepid_x"]
cfpTable = cfpTable.drop(["kepid_x", "kepid_y"], axis=1)


Now merge with the stellar table, using "kepid" as the key

In [6]:
cfpTable = pd.merge(cfpTable, kic, on="kepid", how="inner")


In [7]:
len(cfpTable)

8054

Now cfpTable contains everything we want to know about each entry

Let's add Galactic coordinates

In [8]:
c = SkyCoord(ra=cfpTable.ra*u.degree, dec=cfpTable.dec*u.degree, frame='icrs')
galLong = np.array(c.galactic.l)
galLat = np.array(c.galactic.b)
cfpTable["galLong"] = galLong
cfpTable["galLat"] = galLat


In [9]:
list(cfpTable)

['rowid',
 'kepoi_name',
 'fpwg_koi_period',
 'fpwg_disp_status',
 'fpwg_disp_source',
 'fpwg_disp_eb',
 'fpwg_disp_offst',
 'fpwg_disp_perep',
 'fpwg_disp_other',
 'fpwg_flux_ss',
 'fpwg_flux_slflum',
 'fpwg_flux_oedp',
 'fpwg_flux_vshape',
 'fpwg_flux_depth',
 'fpwg_flux_dilutn',
 'fpwg_flux_lcurve',
 'fpwg_flux_ootvar',
 'fpwg_flux_corvar',
 'fpwg_flux_other',
 'fpwg_flux_stellr',
 'fpwg_flux_noteb',
 'fpwg_offst_sig',
 'fpwg_offst_badim',
 'fpwg_offst_inval',
 'fpwg_offst_vis',
 'fpwg_offst_star',
 'fpwg_offst_qvars',
 'fpwg_offst_other',
 'fpwg_offst_nooff',
 'fpwg_perep_match',
 'fpwg_perep_direc',
 'fpwg_perep_col',
 'fpwg_perep_ghost',
 'fpwg_perep_video',
 'fpwg_perep_ref',
 'fpwg_fa_starvar',
 'fpwg_fa_unique',
 'fpwg_fa_thermal',
 'fpwg_fa_ntl',
 'fpwg_fa_artifact',
 'fpwg_fa_other',
 'fpwg_fa_notfa',
 'fpwg_fop_rvs1',
 'fpwg_fop_rvs2',
 'fpwg_fop_rvs3',
 'fpwg_fop_dblline',
 'fpwg_fop_imexam',
 'fpwg_fop_imblend',
 'fpwg_efp_public',
 'fpwg_efp_pubexam',
 'fpwg_efp_accept',

In [10]:
cfpTable.to_csv("cfpTable.csv")